In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

mps


In [2]:
with open("wizard_of_oz.txt", "r", encoding = "utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)


['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [3]:
# character level tokenizers, initialize the encoder & decoder
string_to_int = { ch:i for i, ch in enumerate(chars) }    # for loops
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l : ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]    # 訓練佔80%
validation_data = data[n:]    # 評估佔20%


# Get batches
def get_Batch(split):
    data = train_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_Batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[72, 73, 71, 54, 73, 62, 68, 67],
        [71, 65, 11,  1,  3, 32, 58,  5],
        [ 1, 73, 68, 65, 57,  0, 74, 72],
        [71, 54, 55, 65, 58,  1, 68, 59]], device='mps:0')
targets:
tensor([[73, 71, 54, 73, 62, 68, 67, 52],
        [65, 11,  1,  3, 32, 58,  5, 72],
        [73, 68, 65, 57,  0, 74, 72,  1],
        [54, 55, 65, 58,  1, 68, 59,  1]], device='mps:0')


In [5]:

# x = train_data[:block_size]
# y = train_data[1:block_size+1]

# for t in range(block_size):
#     context = x[:t+1]
#     target = y[t]
#     print('When input is', context, 'target is', target)

In [6]:
 # we are only reporting the losses, just to make sure it's not using any gradients here
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_Batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
# Initialize neural net, Logits and Reshaping
# Generate function and give the model some context
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # pytorch documentation requires 'shape' to be -> (N, C). N is the batches and C is the channel
            # So we want B, C, T. We can combine B(batch) & T(time dimension) into N. Simply, multiply two peramerters
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    # generate token
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for i in range(max_new_tokens):
            # get the prediction
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :]    # become (B, C)
            # apply softmax to get probability
            probs = F.softmax(logits, dim=-1)   # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs,num_samples=1)    # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=-1)    # (B, T+1)
        return index


model = BigramLanguageModel(vocab_size)
m = model.to(device)


context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


[T851WZL*0Vk'AI9bQOH-,y]T-]K_8H1l.aBWv.ALxZT6uhC'u,xS:AKsB?jC* e-[Gazg;OQv&s7Jc"jx*SfHpst0tmkHQ)s7x&XY
XV0jxDUL85Q[y2.s3?4Pwp:
).5jCyutHeSj3I9iS;*,?2D00xX.q(FA3plXKBfg]mOPwi85P8
)t-;_5I5J_TLtggo1Dpcd785)tO1JmBU3lC!rx4p-oTrslskN Ji6mi]FI5jkzOnfIQhQcB-I2lb'o&jZbBRy?6
PXBacAAfI[I!m9(.4):g!cR-)4&X2N?r&f!'q2nWg8t!;QM?2zjb5y2.5o;IbSf,)QB;n_cXv8Pw?]Hei9ycteR9t7a'o[T6:Wij3m0H-?5
MAh4QHcA,l75ukvhqu,9cZ_J m:gMw(;Ozjc:t&3'&3k"Ir[Cy?94C;Oq;II9.m: J-c5Qeh0cXcTQN
d"-'6BVOr3WAn
h_K5Gx_k?xv1Z1qI9c]PwW:rBxXRLtv8


In [8]:
# create Pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# Standard training loop architecture for basic model
# this is the training loop
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
        
    # sample a batch of data
    xb, yb = get_Batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)    # to make sure that the previous gradient does not add up the current one -> don't want accumulative gradients. Because the previous gradients are from previous data, sometimes those might be bias.
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.889, val loss: 4.887
step: 250, train loss: 4.841, val loss: 4.808
step: 500, train loss: 4.791, val loss: 4.784
step: 750, train loss: 4.716, val loss: 4.681
4.937317848205566


In [9]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ry9!gcT8ApN
m*lT81_H1_NEV22222.L&"?Rsv9"ctv6uyP'6kPAZl o3'[tbY7xoUEst]uz*iuzjbyNPwen)zQ_zf'SfI1qAdFNBSjtx&'_nXv h_HNL*Gh3n"hO5Afzjm922Et6s-lQ(_J_PjyP'zHZHpy?Z30iWu*,O,(v8B-r[em0.*q8xpI[.:SfrCAG!1Mb;QD[(;Zza9
mQ[XylwQDJE[
OE4-dW.MANrAPVcluImZb?k"b'AAnBdZ(!v1)
)fq:Mc"WIoUZUEm01SnB.&37wdeI"Br?]NLFp!a,';zueFxpwm6)okZLTMC!veteXq87isj[C0A_6a_y?Rhex0PVjcP5iyq]fz&qJ()s?vB.m?.*RxQrzq"C*]TYBr?2"G-V853Zm't6KD,75Gv53OrBQA5QNSd.*Nqy.7mk:gQJ,8BakIruZwW?p;VMAd34*]063nRp)I"n]p-4!NF,'p)JQyG Jdd1fSXMV0FG hYEtUEN]
